In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Read train data
train_data = pd.read_csv('./data/train.csv')

In [39]:
# Explatory data analysis
train_data.head()
train_data.shape
train_data.ndim
train_data.columns
train_data.info()
train_data.Survived.value_counts()
train_data.Pclass.value_counts()
train_data.Sex.value_counts()
train_data.SibSp.value_counts()
train_data.Parch.value_counts()
train_data.Embarked.value_counts()
train_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [45]:
# Data cleaning

def get_nan(data, feature):
    return data[data[feature].isnull()]

for feature in train_data.columns:
    print('{0}: {1}'.format(feature, len(get_nan(train_data, feature))))

# As we can see from the results, we have some missing data for Age, Cabin and Embarked.

PassengerId: 0
Survived: 0
Pclass: 0
Name: 0
Sex: 0
Age: 177
SibSp: 0
Parch: 0
Ticket: 0
Fare: 0
Cabin: 687
Embarked: 2


In [43]:
# sns.countplot(x='Fare', hue='Survived', data=train_data)
len(train_data)

891

In [5]:
from sklearn.model_selection import train_test_split

features = ['Pclass', 'Sex', 'SibSp', 'Parch']
train_data = train_data.replace({'male': 0, 'female': 1})
feature_data = train_data[features]
label_data = train_data.Survived
features_train, features_test, labels_train, labels_test = train_test_split(feature_data, label_data, test_size=0.3, random_state=42)

In [ ]:
train_data[train_data.isnull().any(axis=1)]

In [6]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(features_train, labels_train)
predictions = model.predict(features_test)

In [8]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(labels_test, predictions)
print(accuracy)

0.7910447761194029
